In [ ]:
# For Google Colab use
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd '/content/drive/MyDrive/Colab Notebooks/MInf-DeepfakeDetection-FrameDifferencing'    
except ModuleNotFoundError:
    pass

In [ ]:
# Imports
from __future__ import division

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from numpy.random import seed
from time import time
from pathlib import Path

import tensorflow as tf

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, Model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop, SGD
from keras.applications import Xception
from keras import metrics
from keras.losses import BinaryCrossentropy
from keras import backend as K

# !pip install tensorflow_addons;
# from tensorflow_addons.losses import SigmoidFocalCrossEntropy

# !pip install -U keras-tuner;
from kerastuner.tuners import Hyperband
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner import Objective

# !pip install face_recognition;
# from Util import pipeline
from Util import experiment_tuner_models
from Baseline.classifiers import *

In [ ]:
def evaluate_model(TEST_MODEL, EXPERIMENT_NAME:str, TEST_DIR:str, BACTH_SIZE:int=64, \
                   IMG_HEIGHT:int=256, IMG_WIDTH:int=256, DATA_GENERATOR_SEED:int=1337, \
                   HISTORY=None):
    
    from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
    
    EXPERIMENT_NAME = EXPERIMENT_NAME.replace(" ", "_")
    EXPERIMENT_FOLDER_NAME = f'./Results/{EXPERIMENT_NAME}'
    Path(EXPERIMENT_FOLDER_NAME).mkdir(parents=True, exist_ok=True)

    # Define testing datagen
    print('Defining testing datagen...')
    TEST_DATAGEN = ImageDataGenerator(rescale=1./255)
    TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(directory = TEST_DIR,
                                                      batch_size = BACTH_SIZE,
                                                      class_mode = 'binary', 
                                                      target_size =  (IMG_HEIGHT, IMG_WIDTH),                                
                                                      seed = DATA_GENERATOR_SEED,
                                                      follow_links = True)

    # Evaluate test set and get metrics, e.g. loss
    print('Evaluating test set and getting metrics...')
    TEST_HISORY = TEST_MODEL.evaluate(TEST_GENERATOR,
                                      batch_size = BACTH_SIZE,
                                      return_dict = True)
    TEST_GENERATOR.reset()

    # Get predictions and ground truths
    print('Predicting on test set...')
    Y_pred = (TEST_MODEL.predict(TEST_GENERATOR)).ravel()
    Y_true = (TEST_GENERATOR.classes).ravel()

    # Plot AUC and save plot
    print('Plotting AUC...')
    fpr, tpr, _ = roc_curve(Y_true, Y_pred)
    try:
        roc_auc = auc(fpr, tpr)
    except:
        roc_auc = None
        roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='magenta', lw=2, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.01])
    plt.xlabel('False Positive Rate (Fake Incorrectly Classed Rate)')
    plt.ylabel('True Positive Rate (Fake Correctly Classed Rate)')
    plt.title(f'{EXPERIMENT_NAME.replace("_", " ")} ROC')
    plt.legend(loc="lower right")
    print(f'Saving AUC to {EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_AUC.pdf')
    plt.savefig(f'{EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_AUC.pdf')

    # Create file, write metrics
    print(f'Saving summary file to {EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_summary.txt')
    summary_file = open(f'{EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_summary.txt', 'w')
    summary_file.write(str(TEST_HISORY).replace(', \'', ':\n\n'))
    print(str(TEST_HISORY).replace(', \'', ':\n'))

    # find classification thershold which results in max auc
    roc_aucs = []
    for thrshld in map(lambda x: x/1000.0, range(0, 1000+1)):
        fpr, tpr, _ = roc_curve(Y_true, (Y_pred >= thrshld).astype(int))
        roc_aucs.append((auc(fpr, tpr), thrshld))
    max_auc_thrshld = max(roc_aucs)[1]

    # use max auc classification thershold to get confusion matrix and classification report
    y_pred = (Y_pred >= max_auc_thrshld).astype(int)
    summary_output_text = f'''\n\nTHRESHOLD = {max_auc_thrshld}
    \nCONFUSION MATRIX\n{confusion_matrix(Y_true, y_pred)}
    \nCLASSIFICATION REPORT\n{classification_report(Y_true, y_pred, target_names = ["REAL", "FAKE"])}
    _____________________________________________________\n'''
    summary_file.write(summary_output_text)
    summary_file.write(f'''\n{list(zip(Y_true, Y_pred))}''')
    print(summary_output_text)
    summary_file.close()

    # plot history (of training), if provided
    if HISTORY is not None:
        print('Plotting training history x4 plots...')
        acc = HISTORY.history['acc']
        auc = HISTORY.history['auc']
        loss = HISTORY.history['loss']
        fp = HISTORY.history['fp']

        val_acc = HISTORY.history['val_acc']
        val_auc = HISTORY.history['val_auc']
        val_loss = HISTORY.history['val_loss']
        val_fp = HISTORY.history['val_fp']

        epochs = range(len(acc))
        fig, axs = plt.subplots(2, 2, figsize=(10,10))
        fig.suptitle(f'{EXPERIMENT_NAME.replace("_", " ")} Train & Validation Metrics')

        axs[0, 0].plot(epochs, acc, 'r', label='Train Binary Accuracy')
        axs[0, 0].plot(epochs, val_acc, 'b', label='Validation Binary Accuracy')
        axs[0, 0].set_title('Binary Accuracy')
        axs[0, 0].set_ylabel('Accuracy')
        axs[0, 0].set_xlabel('Epochs')
        axs[0, 0].legend()

        axs[0, 1].plot(epochs, loss, 'r', label='Train Loss')
        axs[0, 1].plot(epochs, val_loss, 'b', label='Validation Loss')
        axs[0, 1].set_title('Loss')
        axs[0, 1].set_ylabel('Loss')
        axs[0, 1].set_xlabel('Epochs')
        axs[0, 1].legend()

        axs[1, 0].plot(epochs, auc, 'r', label='Train AUC')
        axs[1, 0].plot(epochs, val_auc, 'b', label='Validation AUC')
        axs[1, 0].set_title('AUC')
        axs[1, 0].set_ylabel('AUC')
        axs[1, 0].set_xlabel('Epochs')
        axs[1, 0].legend()

        axs[1, 1].plot(epochs, fp, 'r', label='Train False Positives')
        axs[1, 1].plot(epochs, val_fp, 'b', label='Validation False Positives')
        axs[1, 1].set_title('False Positives')
        axs[1, 1].set_ylabel('FP')
        axs[1, 1].set_xlabel('Epochs')
        axs[1, 1].legend()
        
        print(f'Saving history plots to {EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_history.pdf')
        fig.savefig(f'{EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_history.pdf')
    
    return Y_true, Y_pred

In [ ]:
# General model settings
# TEST_MODEL = MesoInception4().model
# WEIGHTS_PATH = './Baseline/weights/MesoInception_F2F.h5'
# TEST_MODEL.load_weights(WEIGHTS_PATH)
# TEST_DIR = './Celeb-DF-v2/Celeb-rnd'

In [ ]:
for df_name, df_type in tqdm([('Random_Frame', 'rnd'), ('Frame_Averaging', 'avg'), ('Frame_Difference', 'diff')]):
        for weight_path in [f'./Baseline/weights/MesoInception_F2F.h5', f'./Baseline/weights/MesoInception_DF.h5']:
            weight_name = weight_path.split('_')[-1].replace('.h5', '')
            print(df_type.upper(), weight_name.upper())

            TEST_MODEL = None
            TEST_MODEL = MesoInception4().model
            TEST_MODEL.load_weights(weight_path)

            evaluate_model(TEST_MODEL = TEST_MODEL,
                           EXPERIMENT_NAME = f'Untrained_MesoInception4_({weight_name})_on_{df_name}_Dataset',
                           TEST_DIR = f'./Celeb-DF-v2/Celeb-{df_type}-30-test')

In [ ]:
# y_true, y_pred = evaluate_model(TEST_MODEL = TEST_MODEL, 
#                                 EXPERIMENT_NAME = "test_1", 
#                                 TEST_DIR = TEST_DIR)